# Libbitcoin BX: Transactions with absolute locktime.
In this BX tutorial, we will send funds to a `checklocktimeverify` encumbered output, and spend this after the absolute locktime has expired.
<img src="images/cltv_overview.jpg" alt="drawing" style="" width="700px"/>

<hr style="border: 0.5px dashed #000;">

## 1) Setup wallet and test funds
### 1.1) Generate private/public key pair
#### 1.1.1) New private key

In [1]:
bx seed | bx ec-new

84677322ca7a52a103ec8f1cae4b9910564e645521cc4cff189dee5a8ec98029


#### 1.1.2) Save your private key

In [2]:
echo e60c1ee5fb938aa041c15594b0df6ffbedbfc33e4aaf231b7eb4dbf18c464d8e

e60c1ee5fb938aa041c15594b0df6ffbedbfc33e4aaf231b7eb4dbf18c464d8e


#### 1.1.3) Derive public key<br>

In [3]:
bx ec-to-public e60c1ee5fb938aa041c15594b0df6ffbedbfc33e4aaf231b7eb4dbf18c464d8e

02e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31


#### 1.1.4) Derive public key hash

In [4]:
bx sha256 02e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31 | bx ripemd160

42a8bcd5f61fa17adbe800de6382f24bd1db0d7f


### 1.2) Derive testnet address
Add version prefix for mainnet/testnet<br>
* `6f` in hex
* `111` in decimal

And base58 encode result

In [5]:
bx wrap-encode -v 111 42a8bcd5f61fa17adbe800de6382f24bd1db0d7f \
| bx base58-encode


mmbR2p2QQULJBKBHoEFYDByjdSV9WhKJ3Q


### 1.3) Send test coins funds to your testnet address
Note your spendable p2pkh UXTO's below...

* UXTO 0: 
    * TXID: `d6c4f5ea730a5e35ef126510a5542b01b7013d7601a5d741301348ef3cfae870`<br>
    * Index: `1`
    * Amount: `130000000`
* UXTO 1: 
    * TXID: `a9a02b38c5b2513226c9a22bd636d35b7fcff87ace084a9893c22838fe2e8d94`<br>
    * Index: `1`
    * Amount: ``

<hr style="border: 0.5px dashed #000;">

## 2) Spending to a CLTV encumbered output<br>
<img src="images/spend_to_cltv.jpg" alt="drawing" style="" width="700px"/>

### 2.1) Build transaction with CLTV encumbered output for signing

#### 2.1.1) Determine absolute locktime in blockheight
* `$ bx fetch-height`
* `$ expr [current blockheight] + [delay in blocks]`.
* Encode locktime in hex (little endian)

In [6]:
bx fetch-height 

1407354


In [7]:
expr 1384964 + 4

1384968


In [8]:
printf '%x\n' 1384968

152208


**Note:** Don't forget to flip byte-order (4-Bytes) above for output script below.

#### 2.1.2) Encode CLTV-encumbered P2PKH output script
* **`[locktime]`**
* **`CHECKLOCKTIMEVERIFY`**
* `DROP`
* `DUP`
* `HASH160`
* `[public key hash]`
* `EQUALVERIFY`
* `CHECKSIG`

In [1]:
bx script-encode \
"[08221500] CHECKLOCKTIMEVERIFY DROP DUP HASH160 [42a8bcd5f61fa17adbe800de6382f24bd1db0d7f] EQUALVERIFY CHECKSIG"

0408221500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac


#### 2.1.3) Encode output amount
* Assume `700 satoshi` transaction fee
* Output amount = `[UXTO spendable amount]`-`700`

In [10]:
expr 130000000 - 700

129999300


#### 2.1.4) Encode transaction for signing
* `bx tx-encode -i [prevout txid:index:sequence] -o [output script:amount]`
* Input sequence defaults to `ffffffff`/`4294967295`

In [11]:
bx tx-encode \
-i d6c4f5ea730a5e35ef126510a5542b01b7013d7601a5d741301348ef3cfae870:1:4294967295 \
-o 04fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac:129999300

010000000170e8fa3cef48133041d7a501763d01b7012b54a5106512ef355e0a73eaf5c4d60100000000ffffffff01c4a1bf07000000002004fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000


### 2.2) Building the input script

#### 2.2.1) Signing the transaction with the private key
* `bx fetch-tx` previous output script
* `bx input-sign` `[private key]` `["previous output script"]` `[encoded transaction]`
    * Outputs DER encoded signature

In [12]:
bx fetch-tx -f json d6c4f5ea730a5e35ef126510a5542b01b7013d7601a5d741301348ef3cfae870 \
| jq ".transaction.outputs[1].script"

"dup hash160 [42a8bcd5f61fa17adbe800de6382f24bd1db0d7f] equalverify checksig"


In [13]:
bx input-sign \
e60c1ee5fb938aa041c15594b0df6ffbedbfc33e4aaf231b7eb4dbf18c464d8e \
"dup hash160 [42a8bcd5f61fa17adbe800de6382f24bd1db0d7f] equalverify checksig" \
010000000170e8fa3cef48133041d7a501763d01b7012b54a5106512ef355e0a73eaf5c4d60100000000ffffffff01c4a1bf07000000002004fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000

3045022100987522f31b627a5af943bba86be002fa980841a2b4796964c50fa6009cc44bd502202a0ccf99a93d33055dff5586898b3ca1733889f1ee322ec5f01363cb796491c501


#### 2.2.2) Set the input script into the finalised transaction
* The input script to spend the previous p2pkh output script is `[endorsement]` `[public key]`
* `bx input-set` `"input script"` `[encoded transaction]`

In [14]:
bx input-set \
"[3045022100987522f31b627a5af943bba86be002fa980841a2b4796964c50fa6009cc44bd502202a0ccf99a93d33055dff5586898b3ca1733889f1ee322ec5f01363cb796491c501] [02e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31]" \
010000000170e8fa3cef48133041d7a501763d01b7012b54a5106512ef355e0a73eaf5c4d60100000000ffffffff01c4a1bf07000000002004fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000

010000000170e8fa3cef48133041d7a501763d01b7012b54a5106512ef355e0a73eaf5c4d6010000006b483045022100987522f31b627a5af943bba86be002fa980841a2b4796964c50fa6009cc44bd502202a0ccf99a93d33055dff5586898b3ca1733889f1ee322ec5f01363cb796491c5012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31ffffffff01c4a1bf07000000002004fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000


### 2.3) Validate & Broadcast the endorsed transaction
* `bx validate-tx` `[encoded transaction]`
* `bx tx-decode -f json` `[encoded transaction]` | `jq ".transaction.hash"`

In [15]:
bx validate-tx \
010000000170e8fa3cef48133041d7a501763d01b7012b54a5106512ef355e0a73eaf5c4d6010000006b483045022100987522f31b627a5af943bba86be002fa980841a2b4796964c50fa6009cc44bd502202a0ccf99a93d33055dff5586898b3ca1733889f1ee322ec5f01363cb796491c5012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31ffffffff01c4a1bf07000000002004fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000

matching transaction with unspent outputs


: 255

In [ ]:
bx tx-decode -f json \
010000000170e8fa3cef48133041d7a501763d01b7012b54a5106512ef355e0a73eaf5c4d6010000006b483045022100987522f31b627a5af943bba86be002fa980841a2b4796964c50fa6009cc44bd502202a0ccf99a93d33055dff5586898b3ca1733889f1ee322ec5f01363cb796491c5012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31ffffffff01c4a1bf07000000002004fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000 \
|jq ".transaction.hash"

In [ ]:
bx send-tx 010000000170e8fa3cef48133041d7a501763d01b7012b54a5106512ef355e0a73eaf5c4d6010000006b483045022100987522f31b627a5af943bba86be002fa980841a2b4796964c50fa6009cc44bd502202a0ccf99a93d33055dff5586898b3ca1733889f1ee322ec5f01363cb796491c5012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31ffffffff01c4a1bf07000000002004fa211500b17576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac00000000

<hr style="border: 0.5px dashed #000;">

## 3) Spend the CLTV encumbered UXTO

<img src="images/spend_from_cltv.jpg" alt="drawing" style="" width="700px"/>

### 3.1) Build transaction with activated locktime for signing

#### 3.1.1) Encode p2pkh output script

In [3]:
bx script-encode "DUP HASH160 [42a8bcd5f61fa17adbe800de6382f24bd1db0d7f] EQUALVERIFY CHECKSIG"

76a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac


**3.1.2) Encode transaction with locktime & set input sequence**<br>
* Reference Previous Outpoint<br>
* Transaction locktime field must be `>=` locktime in CLTV output being spent.
* Input sequence msut be less than maximum, so `fffffffe`/`4294967294`

In [ ]:
bx tx-encode \
-l 1384968 \
-i 918c9e1acf52698ea328df65c134ba1c5fdd119aea476ca6b934bbd03a6e3596:0:4294967294 \
-o 7576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac:129998600


### 3.2) Building the input script

#### 3.2.1)  Signing the transaction with the private key
* `bx fetch-tx` previous output script
* `bx input-sign` `[private key]` `["previous output script"]` `[encoded transaction]`
    * Outputs DER encoded signature

In [ ]:
bx fetch-tx -f json 918c9e1acf52698ea328df65c134ba1c5fdd119aea476ca6b934bbd03a6e3596 \
| jq ".transaction.outputs[0].script"

In [ ]:
bx input-sign e60c1ee5fb938aa041c15594b0df6ffbedbfc33e4aaf231b7eb4dbf18c464d8e \
"[fa211500] CHECKLOCKTIMEVERIFY DROP DUP HASH160 [42a8bcd5f61fa17adbe800de6382f24bd1db0d7f] EQUALVERIFY CHECKSIG" \
010000000196356e3ad0bb34b9a66c47ea9a11dd5f1cba34c165df28a38e6952cf1a9e8c910000000000feffffff01089fbf07000000001a7576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac08221500


#### 3.2.2) Set the input script into the finalised transaction

In [ ]:
bx input-set "[3045022100c7a966f9eba206a6a59220defb4dc1801171bf5be00dff9fa5f5244e7678ed900220055a61e6e9b100eb57238fb6e7a214dbb53a1e459617c6ba48c71daacd6b0cc101] [02e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31]" \
010000000196356e3ad0bb34b9a66c47ea9a11dd5f1cba34c165df28a38e6952cf1a9e8c910000000000feffffff01089fbf07000000001a7576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac08221500


### 3.3) Validate & Broadcast the endorsed transaction
* `bx validate-tx` `[encoded transaction]`
* Get TXID: `bx tx-decode -f json` `[encoded transaction]` | `jq ".transaction.hash"`
* `bx send-tx` `[encoded transaction]`

In [ ]:
bx validate-tx \
010000000196356e3ad0bb34b9a66c47ea9a11dd5f1cba34c165df28a38e6952cf1a9e8c91000000006b483045022100c7a966f9eba206a6a59220defb4dc1801171bf5be00dff9fa5f5244e7678ed900220055a61e6e9b100eb57238fb6e7a214dbb53a1e459617c6ba48c71daacd6b0cc1012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31feffffff01089fbf07000000001a7576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac08221500


In [ ]:
bx tx-decode -f json \
010000000196356e3ad0bb34b9a66c47ea9a11dd5f1cba34c165df28a38e6952cf1a9e8c91000000006b483045022100c7a966f9eba206a6a59220defb4dc1801171bf5be00dff9fa5f5244e7678ed900220055a61e6e9b100eb57238fb6e7a214dbb53a1e459617c6ba48c71daacd6b0cc1012102e1af2f9086e95728bc3c768ee3e83ab81dd1841b92e281caa9f92501fd9b2b31feffffff01089fbf07000000001a7576a91442a8bcd5f61fa17adbe800de6382f24bd1db0d7f88ac08221500 \
| `jq ".transaction.hash"`
